In [1]:
from abc import ABC
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.utils.data as Data
import torch

In [2]:
# 手写数字训练数据集
mnist = np.load("../../../../Other/datasets/NumPy/mnist.npz")
X_train, y_train, X_test, y_test = torch.unsqueeze(torch.tensor(mnist['x_train']), 1).to(dtype=torch.float32),
                                   torch.tensor(mnist['y_train']).to(dtype=torch.long),
                                   torch.unsqueeze(torch.tensor(mnist['x_test']), 1).to(dtype=torch.float32),
                                   torch.tensor(mnist['y_test']).to(dtype=torch.long)
X_train, X_test = X_train / 255.0, X_test / 255.0

In [3]:
batch_size = 200
train_data = Data.TensorDataset(X_train, y_train)
train_loader = Data.DataLoader(dataset=train_data,
                               batch_size=batch_size,
                               shuffle=True)
train_X = Data.DataLoader(X_train, batch_size=batch_size)
test_X = Data.DataLoader(X_test, batch_size=batch_size)

In [4]:
class LeNet(nn.Module, ABC):
    def __init__(self):
        super(LeNet, self).__init__()
        self.featrues = nn.Sequential(  # 内部实现了forward函数;各模块顺序执行
            nn.Conv2d(1, 6, 3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(6, 16, 3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(400, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10),  # 10分类问题
        )

    def forward(self, x):
        x = self.featrues(x)
        x = x.reshape(x.size()[0], -1)
        x = self.classifier(x)
        return x

In [5]:
learning_rate = 0.001
criteon = nn.CrossEntropyLoss()  # 定义损失函数为交叉熵
net = LeNet()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lenet = net.to(device)  # 模型设备转移必须在优化器定义前执行

# Adam优化器
optimizer = optim.Adam(net.parameters(),
                       lr=learning_rate,  # 学习率\epsilon;默认lr=0.0001
                       weight_decay=1e-3,  # L2正则化系数,默认weight_decay=0
                       betas=(0.9, 0.999),  # \rho_1, \rho_2;默认betas=(0.9, 0.999)
                       eps=1e-8)  # 分母中的小常数\delta;默认eps=1e-8

In [6]:
from torch_model.train_evaluate import Train_Evaluate

t_and_v = Train_Evaluate(model=lenet, optimizer=optimizer, criterion=criteon, epochs=5)

In [7]:
metric_reslut = t_and_v.train_eval(train_loader=train_loader, valid_sets=[(train_X, y_train), (test_X, y_test)])
metric_reslut  # 每一轮验证数据集的损失值

C:\Users\duanm\anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Train Epoch: 0  [0    /60000 (0  %)]	Loss: 2.304474
Train Epoch: 0  [2000 /60000 (3  %)]	Loss: 2.264626
Train Epoch: 0  [4000 /60000 (7  %)]	Loss: 2.102553
Train Epoch: 0  [6000 /60000 (10 %)]	Loss: 1.610993
Train Epoch: 0  [8000 /60000 (13 %)]	Loss: 1.025231
Train Epoch: 0  [10000/60000 (17 %)]	Loss: 0.734917
Train Epoch: 0  [12000/60000 (20 %)]	Loss: 0.622379
Train Epoch: 0  [14000/60000 (23 %)]	Loss: 0.402567
Train Epoch: 0  [16000/60000 (27 %)]	Loss: 0.434148
Train Epoch: 0  [18000/60000 (30 %)]	Loss: 0.458251
Train Epoch: 0  [20000/60000 (33 %)]	Loss: 0.377119
Train Epoch: 0  [22000/60000 (37 %)]	Loss: 0.387250
Train Epoch: 0  [24000/60000 (40 %)]	Loss: 0.436786
Train Epoch: 0  [26000/60000 (43 %)]	Loss: 0.435575
Train Epoch: 0  [28000/60000 (47 %)]	Loss: 0.411630
Train Epoch: 0  [30000/60000 (50 %)]	Loss: 0.301655
Train Epoch: 0  [32000/60000 (53 %)]	Loss: 0.376645
Train Epoch: 0  [34000/60000 (57 %)]	Loss: 0.243595
Train Epoch: 0  [36000/60000 (60 %)]	Loss: 0.354421
Train Epoch:

[[0.20348581671714783, 0.18865929543972015],
 [0.11217640340328217, 0.10300183296203613],
 [0.08679686486721039, 0.07907784730195999],
 [0.08009027689695358, 0.07865934818983078],
 [0.07080507278442383, 0.06784366071224213]]